<a href="https://colab.research.google.com/github/Shiwani123693/24B0431_SOC_FINAL_PROJECT_2025/blob/main/AI_DS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df = pd.read_csv('/content/netflix_titles.csv')
display(df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# 1. Visualization of dataset

### Customize and display chart Subtask:
 ## Add titles, labels, and any other necessary customizations to the sunburst chart and display it.

In [6]:
genres_split = df['listed_in'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
genres_df = genres_split.to_frame(name='genre')
genres_df = genres_df.merge(df[['show_id', 'type']], left_index=True, right_index=True)
genre_type_counts = genres_df.groupby(['type', 'genre']).size().reset_index(name='count')
import plotly.express as px

fig = px.sunburst(genre_type_counts, path=['type', 'genre'], values='count', title='Netflix Content Distribution by Type and Genre')
fig.show()

### vertical bar chart showing the distribution of genres on Netflix.
#### The x-axis represents different genres,

 #### The y-axis represents the number of movies/shows in each genre.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt


genres = df['listed_in'].str.split(', ', expand=True).stack()
genre_counts = genres.value_counts().sort_values(ascending=False)

plt.figure(figsize=(15, 8)) # Increased figure size
genre_counts.plot(kind='bar')
plt.xlabel("Genres")
plt.ylabel("Number of Movies/Shows")
plt.title("Distribution of Genres on Netflix")
plt.xticks(rotation=90) # Increased rotation angle
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

# 2. Descriptive statistics

### a.) Setup and Numerical Statistics
#### This section loads the data and generates summary statistics for numerical features, like release_year.

In [8]:
import pandas as pd
import numpy as np



# --- 1. Summary Statistics for Numerical Data ---

# A. Descriptive Statistics for 'release_year'
# Measures of central tendency (mean, median), dispersion (std, min, max, range), and shape (skewness).
release_year_stats = df['release_year'].describe().to_frame()
release_year_stats.loc['range'] = release_year_stats.loc['max'] - release_year_stats.loc['min']
release_year_stats.loc['skewness'] = df['release_year'].skew()
release_year_stats = release_year_stats.round(2)

print("--- Descriptive Statistics for 'release_year' ---")
print(release_year_stats)

## b.) Categorical and Target Feature Statistics
### This section calculates the frequency of key categorical features, including your target feature, listed_in (Genre), and others like type and rating

In [ ]:
# --- 2. Frequency and Mode for Key Categorical Data ---

# A. Frequency of 'type' (Movie vs. TV Show)
# Calculate type counts from the original DataFrame before dropping rows
type_counts = df['type'].value_counts().to_frame()
type_counts['percentage'] = (type_counts['count'] / len(df) * 100).round(2)
print("\n--- Frequency and Percentage for 'type' ---")
print(type_counts)

# B. Mode and Top 5 Frequencies for 'rating'
rating_mode = df['rating'].mode()[0]
rating_counts = df['rating'].value_counts().head(5).to_frame()
print(f"\nMode for 'rating': {rating_mode}")
print("\nTop 5 Frequencies for 'rating' (Content Ratings) ---")
print(rating_counts)

# C. Mode and Top 5 Frequencies for 'country' (Handling Multi-Country)
# Take only the first country listed for a simpler analysis
df_temp = df.copy()
df_temp['primary_country'] = df_temp['country'].str.split(',').str[0].str.strip()

country_mode = df_temp['primary_country'].mode()[0]
country_counts = df_temp['primary_country'].value_counts(dropna=True).head(5).to_frame()
print(f"\nMode for 'country': {country_mode}")
print("\nTop 5 Frequencies for 'primary_country' ---")
print(country_counts)

# D. Frequency of Target Feature: 'listed_in' (Genres)
# We use the MultiLabelBinarizer approach to count all individual genres
from sklearn.preprocessing import MultiLabelBinarizer

# Handle missing values for the target feature
df_clean = df.dropna(subset=['listed_in']).copy()
genres = df_clean['listed_in'].str.split(', ')

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(genres)
y_df = pd.DataFrame(y, columns=mlb.classes_)

genre_counts = y_df.sum().sort_values(ascending=False).to_frame(name='count')
genre_counts['percentage'] = (genre_counts['count'] / len(df_clean) * 100).round(2)

print("\n--- Descriptive Statistics for Target Feature: 'listed_in' (Genres) ---")
print(f"Total Unique Genres: {len(mlb.classes_)}")
print("\nTop 5 Most Frequent Genres:")
print(genre_counts.head(5))

# 3.) Distribution of data
### Q-Q plots (Quantile-Quantile plots) is an excellent way to visually assess if your data follows a specific theoretical distribution
### selected two key variables for this analysis
#### release_year: A numerical/time-based feature, checked against Normal (Gaussian) and Uniform distributions.

 ### duration_minutes: A continuous numerical feature (filtered for Movies only), checked against Normal (Gaussian) and Exponential distributions.

In [ ]:
import scipy.stats as stats




release_year = df['release_year'].dropna()

distributions = {
    "Normal": stats.norm,
    "Exponential": stats.expon,
    "Uniform": stats.uniform
}

plt.figure(figsize=(15, 5))
for i, (name, dist) in enumerate(distributions.items(), 1):
    plt.subplot(1, 3, i)
    stats.probplot(release_year, dist=dist, plot=plt)
    plt.title(f"Q-Q Plot vs {name} Distribution")

plt.tight_layout()
plt.show()

# 4. Inferential statistics
### Estimation (4a): Calculates the point estimate (sample mean) and a  95 percent Confidence Interval for the true population mean movie duration.
 ### Hypothesis Test (4b): Performs a one-sample t-test to test if the mean movie duration is significantly different from a hypothesized value of $90$ minutes.

In [ ]:

import scipy.stats as stats



# --- 1. Data Preparation: Define Parameter of Interest (Movie Duration) ---

# Filter for Movies
df_movies = df[df['type'] == 'Movie'].copy()

# Clean 'duration' column to get numerical duration in minutes
df_movies['duration_minutes'] = (
    df_movies['duration']
    .str.replace(' min', '', regex=False)
    .astype(float)
    .dropna()
)

# Use the cleaned numerical duration data
movie_durations = df_movies['duration_minutes'].dropna()

n = len(movie_durations)
sample_mean = movie_durations.mean()
sample_std = movie_durations.std()

# --- 2. Estimation (Task 4a): Point Estimate and Interval Estimate ---

# Point Estimate
point_estimate = sample_mean

# Interval Estimate (95% Confidence Interval for the Population Mean)
confidence_level = 0.95
degrees_freedom = n - 1
standard_error = sample_std / np.sqrt(n)

# Calculate the interval using the t-distribution
confidence_interval = stats.t.interval(
    confidence=confidence_level,
    df=degrees_freedom,
    loc=sample_mean,
    scale=standard_error
)
print(" confidence_interval : ",confidence_interval)

# --- 3. Hypothesis Test (Task 4b): One-Sample t-test ---

# Null Hypothesis (H0): The true mean duration of Netflix movies is 90 minutes (μ = 90)
mu_0 = 90
alpha = 0.05 # Significance level

# Perform the one-sample t-test
t_statistic, p_value = stats.ttest_1samp(movie_durations, popmean=mu_0)

print("t_statistic:",t_statistic)
print("p_value",p_value)
# Determine the conclusion
# ... (Conclusion logic follows below)

## Decision & Conclusion:
### We compare the P-value to the Significance Level ($\alpha = 0.05$).
### Since $0$ is much less than $0.05$, we Reject the Null Hypothesis ($H_0$).
### Conclusion: The Netflix movie duration is statistically significantly different from $90$ minutes. It is definitively longer.

# **(c) Regression Model for a Different Column**
### **A regression model is designed for one specific job: to predict a continuous numerical value**
### but our target column, listed_in, does not contain numbers. It contains categories (or "classes") like "Dramas," "Comedies," and "Documentaries." The correct tool for predicting a category from a predefined list is a classification model.

#### Then our  model predicts a numerical value (0–72) that represents the genre label.This number corresponds to the primary genre of the movie/show.But unlike classification, it predicts a continuous numeric value, not a class directly.

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np



# --- 1. Data Preparation for Regression ---

# A. Create Numerical Target (Y) from 'listed_in'
# 1. Extract the primary genre (first one listed)
df['primary_genre'] = df['listed_in'].apply(lambda x: x.split(',')[0].strip())

# 2. Ordinal Encode the primary genre into a numerical value (0 to 72)
# This is required to force the categorical target into a numerical one for regression.
le = LabelEncoder()
df['genre_code'] = le.fit_transform(df['primary_genre'])
y = df['genre_code']

# B. Prepare Predictors (X): Combine title and description
X = df['title'].fillna('') + ' ' + df['description'].fillna('')

# C. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 2. Model Pipeline Definition ---
model = Pipeline([
    # TF-IDF Vectorizer: Converts text into numerical features
    ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))),
    # Ridge Regression Model
    ('regressor', Ridge(alpha=1.0, random_state=42)),
])

# --- 3. Model Training ---
model.fit(X_train, y_train)

# --- 4. Model Evaluation ---
y_pred = model.predict(X_test)

# Calculate Root Mean Squared Error (RMSE) - The standard metric for regression
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate R-squared (R2) score
r2_score_value = model.score(X_test, y_test)

print(f"Achieved R-squared (R2): {r2_score_value:.4f}")
print(f"Achieved Root Mean Squared Error (RMSE): {rmse:.4f}")

Achieved R-squared (R2): 0.2503
Achieved Root Mean Squared Error (RMSE): 6.4095


#  (d) Classification:
## We perform a first classification model where the model predicts whether the entry is a movie, TV show, or another category and in another the classification model listed_in which is genre of movie predicted .
## In both model we used same type of features title and discription but different target feature for first is type and for other is listed_in which is genre of type

# **1st Classification Model which we predict type of data**

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


# --- 1. Data Preparation (Robust Model - No Data Leakage) ---

# A. Target Variable (Y): Predict the 'type' column ('Movie' or 'TV Show')
y = df['type']

# B. Feature Engineering: ONLY use 'title' and 'description'
# This is done to prevent data leakage from the 'listed_in' column.
df['clean_text_features'] = df['title'].fillna('') + ' ' + df['description'].fillna('')

# C. Define Predictors (X)
X = df['clean_text_features']

# D. Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 2. Model Pipeline Definition ---
model = Pipeline([
    # TF-IDF Vectorizer: Converts text into numerical features
    ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))),
    # LinearSVC: A fast and effective model for classification
    ('clf', LinearSVC(C=1.0, random_state=42, max_iter=10000)),
])

# --- 3. Model Training ---
model.fit(X_train, y_train)

# --- 4. Model Evaluation ---
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Achieved Accuracy: {accuracy * 100:.2f}%")
# 2. PREDICTION FUNCTION
def predict_type(title, description=""):
    new_input = [title + ' ' + description]
    predicted_type = model.predict(new_input)[0]

    print("\n--- PREDICTION ---")
    print(f"Input Title: {title}")
    print(f"Predicted type: {predicted_type}")
    print("------------------")

# 3. YOUR INPUT HERE (Change these values to test your movie/show)
predict_type(
    title="sholay",
    description="Sholay is a 1975 Indian action-adventure film about two petty criminals, Jai and Veeru, hired by a retired police officer to capture the ruthless dacoit Gabbar Singh"
)

Achieved Accuracy: 76.05%

--- PREDICTION ---
Input Title: sholay
Predicted type: Movie
------------------


# **2nd Model of classification which we predict listed_in data**

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
# --- ADDED THESE IMPORTS ---
from sklearn.metrics import accuracy_score, classification_report

# 1. MODEL SETUP (Run this once to train the model)



df['primary_genre'] = df['listed_in'].apply(lambda x: x.split(',')[0].strip())
genre_counts = df['primary_genre'].value_counts()
rare_genres = genre_counts[genre_counts < 2].index
df_filtered = df[~df['primary_genre'].isin(rare_genres)].copy()
df_filtered['text_features'] = df_filtered['title'].fillna('') + ' ' + df_filtered['description'].fillna('')
X = df_filtered['text_features']
y = df_filtered['primary_genre']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('clf', LinearSVC(C=1.0, random_state=42, max_iter=10000)),
])
model.fit(X_train, y_train)


# --- START OF NEW ACCURACY SECTION ---

print("\n--- MODEL ACCURACY ---")
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate and print the overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Model Accuracy: {accuracy * 100:.2f}%\n")

# Print a detailed report (precision, recall, f1-score for each genre)
print("--- Detailed Classification Report ---")
print(classification_report(y_test, y_pred))

# --- END OF NEW ACCURACY SECTION ---


# 2. PREDICTION FUNCTION
def predict_genre(title, description=""):
    new_input = [title + ' ' + description]
    predicted_genre = model.predict(new_input)[0]

    print("\n--- PREDICTION ---")
    print(f"Input Title: {title}")
    print(f"Predicted Genre: {predicted_genre}")
    print("------------------")

# 3. YOUR INPUT HERE (Change these values to test your movie/show)
predict_genre(
    title="sholay",
    description="Sholay is a 1975 Indian action-adventure film about two petty criminals, Jai and Veeru, hired by a retired police officer to capture the ruthless dacoit Gabbar Singh"
)


--- MODEL ACCURACY ---
Overall Model Accuracy: 47.42%

--- Detailed Classification Report ---
                              precision    recall  f1-score   support

          Action & Adventure       0.45      0.51      0.48       172
              Anime Features       0.00      0.00      0.00         4
                Anime Series       0.64      0.20      0.30        35
            British TV Shows       0.19      0.06      0.09        51
    Children & Family Movies       0.56      0.63      0.59       121
           Classic & Cult TV       0.00      0.00      0.00         4
              Classic Movies       0.00      0.00      0.00        16
                    Comedies       0.45      0.52      0.48       242
              Crime TV Shows       0.38      0.30      0.34        80
                 Cult Movies       0.00      0.00      0.00         2
               Documentaries       0.63      0.64      0.63       166
                  Docuseries       0.36      0.20      0.26     